# Dynamic optimization

## Init

In [1]:
# imports
import os #to access system folders
import subprocess #to access ffmpeg in the system
import shutil #to remove directories
import numpy as np #easy vector operations
import math #for operation with infinite
import itertools
from scipy.optimize import curve_fit #fittin of the curve
import json #to handle json files
import matplotlib.pyplot as pl #to display plots
import tkinter as tk #to import file
from tkinter import filedialog #to open import dialog

#constants
PARAM_AVC = {"crfs": 52, "starting_range": [10,41], "lib": "libx264", "container": "mp4", "add_param": ""}
PARAM_HEVC = {"crfs": 52, "starting_range": [15,46], "lib": "libx265", "container": "mp4", "add_param": ""}
PARAM_VP9 = {"crfs": 64, "starting_range": [15,51], "lib": "libvpx-vp9", "container": "webm", "add_param": "-b:v 0"}

#variables - CUSTOM
codec = "avc" #values: "avc", "hevc", "vp9", !!not implemented: "av1", "vvc"
raw_width = 480
raw_height = 270
raw_fps = 29.97
#values "rate","vmaf", "psnr" !!not implemented: "ssim", "mssim"
target_list = {"vmaf": [90, 65], "rate": []}

#input file
root = tk.Tk()
root.withdraw()
source_path = os.path.relpath(filedialog.askopenfilename())
source_name = os.path.basename(source_path).split('.')[0]
REF_PATH = "test_vids/tempRAW_refs/" #raw files for each shot
[os.remove(REF_PATH+f) for f in os.listdir(REF_PATH)] #clean temp_refs folder
DIST_PATH = "test_vids/temp_encoded/" #encoded files for each shot
[shutil.rmtree(DIST_PATH+f) for f in os.listdir(DIST_PATH)] #clean temp_encoded folder

#assessment files path
tm_file = "rd_results/template.json"
rd_file = "rd_results/" + source_name + ".json"
if not os.path.isfile(rd_file): #if file does not exists create it
    with open(rd_file, 'w') as f:
        pass
VMAF_LOGS = "rd_results/vmaf_logs.json"

#shot detection
TIME_LOGS = "shot_detection.log"
shot_th = 0.25 #change shot threshold
num_scenes = 0
duration = 0.0

#output file
OUT_LIST = "shot_list.txt"
OUT_PATH = "test_vids/OPT_vids/"

#all computed points, by row: crf, bitrate, vmaf, psnr
res_matrix = {"crf": None, "bitrate": None, "vmaf": None, "psnr": None}

print("init done")

init done


## Shot change detection

In [2]:
#an empty json structure is generated to be filled and to store computed values
def init_res_matrix(x):
    #res_matrix["crf"] = np.arange(0,x,1).tolist()
    inf_matrix = np.zeros(x)
    #inf_matrix[inf_matrix == 0] = math.inf
    res_matrix["crf"] = inf_matrix.tolist()
    res_matrix["bitrate"] = inf_matrix.tolist()
    res_matrix["vmaf"] = inf_matrix.tolist()
    res_matrix["psnr"] = inf_matrix.tolist()
    
#detect shot changes in the scene and split it into shots
def shot_change_detection(p):
    start_t = 0.0
    end_t = 0.0
    global duration
    #return when the shot changes
    det = f"ffmpeg -i {p} -filter_complex:v \"select='gt(scene,{shot_th})', \
        metadata=print:file={TIME_LOGS}\" -f null -"
    subprocess.call(det, shell=True)
    #get the total duration for the last cut
    idu = f"ffprobe -v error -select_streams v:0 -show_entries format:stream -print_format json {p}"
    dta = json.loads(subprocess.run(idu.split(), stdout=subprocess.PIPE, stderr=subprocess.STDOUT).stdout)
    duration = float(dta['format']['duration'])
    
    with open("shot_detection.log", 'r') as r:
        tm_log = r.read().splitlines()[::2]
    tm_log.append("end pts_time:" + str(duration))
    n = len(tm_log)
    for i,l in enumerate(tm_log): #for each cut
        #create a folder for each scene
        new_dir = str(i)
        new_path = os.path.join(DIST_PATH, new_dir)
        os.mkdir(new_path)
        
        #cut the video
        end_t = l.split("pts_time:",1)[1]
        cut = f"ffmpeg -ss {start_t} -to {end_t} -i {p} \
            -pix_fmt yuv420p {REF_PATH}scene{str(i).zfill(7)}.yuv"
        subprocess.call(cut, shell=True)
        start_t = end_t
    return n

In [3]:
struct_points = [] #structure of target points for the json file
struct_shots = [] #structure of shots for the json file

if source_path.endswith(".yuv"):
    print("yuv input")
elif source_path.endswith(".y4m"):
    print("y4m input")
else:
    print("No such an input type")
    exit()

num_scenes = shot_change_detection(source_path)
    
#init values based on the selected output codec
if codec == "avc":
    s_cod = PARAM_AVC
    init_res_matrix(PARAM_AVC["crfs"])
elif codec == "hevc":
    s_cod = PARAM_HEVC
    init_res_matrix(PARAM_HEVC["crfs"])
elif codec == "vp9":
    s_cod = PARAM_VP9
    init_res_matrix(PARAM_VP9["crfs"])
else:
    print("No such an codec")
    exit()

min_range_crf = s_cod["starting_range"][0]
max_range_crf = s_cod["starting_range"][1]

with open(tm_file, 'r') as f:
    o_data = json.load(f)
    
#add source name and results matrix
o_data["content"] = source_name
o_data["codec"] = codec
o_data["width"] = raw_width
o_data["height"] = raw_height
o_data["fps"] = raw_fps
o_data["shots"][0]["assessment"] = res_matrix
    
#add emplty target points
base_point = o_data["shots"][0]["opt_points"][0]
for t_name in target_list:
    for t_val in target_list[t_name]:
        base_point["metric"] = t_name
        base_point["target"] = t_val
        struct_points.append(base_point.copy())
o_data["shots"][0]["opt_points"] = struct_points
    
#add empty shots
base_shot = o_data["shots"][0]
for i in range(0, num_scenes):
    base_shot["index"] = i #assign index to shots in json file
    struct_shots.append(base_shot.copy())
o_data["shots"] = struct_shots

with open(rd_file, 'w') as w:
    json.dump(o_data, w, separators=(',',': '))

y4m input


ffmpeg version N-106635-g83e1a1de88 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/home/ubuntu/ffmpeg_build --pkg-config-flags=--static --extra-cflags=-I/home/ubuntu/ffmpeg_build/include --extra-ldflags=-L/home/ubuntu/ffmpeg_build/lib --extra-libs='-lpthread -lm' --ld=g++ --bindir=/home/ubuntu/bin --enable-gpl --enable-gnutls --enable-libaom --enable-libass --enable-libfdk-aac --enable-libfreetype --enable-libmp3lame --enable-libopus --enable-libsvtav1 --enable-libdav1d --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libvmaf --enable-version3 --enable-nonfree
  libavutil      57. 24.101 / 57. 24.101
  libavcodec     59. 26.100 / 59. 26.100
  libavformat    59. 22.100 / 59. 22.100
  libavdevice    59.  6.100 / 59.  6.100
  libavfilter     8. 33.100 /  8. 33.100
  libswscale      6.  6.100 /  6.  6.100
  libswresample   4.  6.100 /  4.  6.100
  libpostproc    56.  5.100 / 56.  5.100

## Optimization
Find the shot encoded to a certain crf that has the closest quality or rate to the target

In [4]:
def encode(s,i,c):
    add_info = s_cod["add_param"]
    lib = s_cod["lib"]
    o = DIST_PATH + str(i) + "/" + str(c) + "_" + codec.upper() + "." + s_cod["container"]
    enc = f"ffmpeg -f rawvideo -video_size {raw_width}x{raw_height} -r {raw_fps} \
        -pixel_format yuv420p -i {REF_PATH + s} -c:v {lib} -crf {c} {add_info} {o}"
    subprocess.call(enc, shell=True)
    return o

def assess(s,o):
    c_vmaf = f"ffmpeg -f rawvideo -r {raw_fps} -video_size {raw_width}x{raw_height} -i {REF_PATH + s} \
            -i {o} \
            -lavfi \"[0:v]setpts=PTS-STARTPTS[ref];\
                    [1:v]scale={raw_width}x{raw_height}:flags=bicubic, setpts=PTS-STARTPTS[dist];\
                    [dist][ref]libvmaf=feature=name=psnr:log_path={VMAF_LOGS}:log_fmt=json\" \
            -f null -" #|name=float_ssim|name=float_ms_ssim to compute the other metrics
    subprocess.call(c_vmaf, shell=True)

#store the quality and rate results for each shot at each encoded crf
def store_results(i,c,o):
    with open(VMAF_LOGS, 'r') as r: #extract quality and rate values
        i_data = json.load(r)
    o_data["shots"][i]["assessment"]["crf"][c] = c
    o_data["shots"][i]["assessment"]["vmaf"][c] = i_data["pooled_metrics"]["vmaf"]["mean"]
    o_data["shots"][i]["assessment"]["psnr"][c] = (6*i_data["pooled_metrics"]["psnr_y"]["mean"] + \
        i_data["pooled_metrics"]["psnr_cb"]["mean"] + i_data["pooled_metrics"]["psnr_cr"]["mean"])/8
    info = f"ffprobe -v error -select_streams v:0 -show_entries format:stream -print_format json {o}"
    cout = json.loads(subprocess.run(info.split(), stdout=subprocess.PIPE, stderr=subprocess.STDOUT).stdout)
    o_data["shots"][i]["assessment"]["bitrate"][c] = int(cout["format"]["bit_rate"])
    o_data["shots"][i]["duration"] = float(cout["format"]["duration"])
    
def save_opt(i, t, opt):
    o_data["shots"][i]["opt_points"][t]["crf"] = int(opt)
    with open(rd_file, 'w') as w:
        json.dump(o_data, w, separators=(',',': '))
    
def search_opt(t_name, t_val, sx_end, dx_end, i_sx_end, i_dx_end):
    if t_val < sx_end and t_val > dx_end: #if the target is in the range
        n = interpolate(i_sx_end, i_dx_end)
    elif t_val > sx_end: #if the target is out of the range in the left side
        n = 0 #if no other points in this direction had been stored encode at the min crf
        i = i_sx_end - 1
        while n == 0 and i>= 0:
        #the first point you find is the new lower end of the range
            if not res_matrix[t_name][i] == 0:
                n = interpolate(i, i_sx_end)
            i -= 1
    elif t_val < dx_end: #if the target is out of the range in the right side
        n = s_cod["crfs"]-1 #if there's no other points, encode at the max crf
        i = i_dx_end + 1
        while n == s_cod["crfs"]-1 and i <= s_cod["crfs"]-1:
            #the first point you find is the new upper end of the range
            if not res_matrix[t_name][i] == 0:
                n = interpolate(i_dx_end, i)
            i += 1
    return n

#linear interpolation of the target and the weight alpha between sx and dx
def interpolate(sx, dx):
    alpha = (res_matrix[t_name][sx] - t_val) / (res_matrix[t_name][sx] - res_matrix[t_name][dx])
    new_point = round(res_matrix["crf"][sx] - alpha * (res_matrix["crf"][sx] - res_matrix["crf"][dx]))
    return new_point

def combine():
    print(t_val)
    min_rate = math.inf
    min_dist = 100
    o = []
    for comb in itertools.product(*s_crfs): #for each combination
        dist = 0
        rate = 0
        for i,v in enumerate(comb):
            dist = dist + (100 - o_data["shots"][i]["assessment"]["vmaf"][v]) * o_data["shots"][i]["duration"]
            rate = rate + o_data["shots"][i]["assessment"]["bitrate"][v] * o_data["shots"][i]["duration"]
        dist = dist / duration
        rate = rate / duration
        if t_name == "vmaf":
            if dist < (100 - t_val) and rate < min_rate:
                min_rate = rate
                o = list(comb)
        elif t_name == "rate":
            if rate < t_val and dist < min_dist:
                min_dist = dist
                o = list(comb)
        else:
            print("target error")
            exit()
    return o

In [5]:
point_index = 0
target_index = 0
for t_name in target_list:
    for t_val in target_list[t_name]:
        
        if True: #full search
            shot_index = 0
            if target_index == 0:
                s_crfs = np.zeros(shape=(num_scenes, max_range_crf-min_range_crf), dtype=int) #init structure
                for shot in sorted(os.listdir(REF_PATH)): #for each shot
                    for current_crf in range(min_range_crf, max_range_crf):
                        out = encode(shot, shot_index, current_crf) #encoding
                        assess(shot, out) #quality assessment
                        store_results(shot_index, current_crf, out)
                    s_crfs[shot_index] = [c for c in o_data["shots"][i]["assessment"]["crf"] if c != 0] #take only encoded shots
                    shot_index += 1
            opt_crfs = combine() #create all combinations
            for i in range(0,num_scenes): #save the opt crf for each shot
                save_opt(i, target_index, opt_crfs[i])
            
        else: #lagrange
            slope = None
            slope_range = [0.0, 0.0]
            lh_slope = 0.0
            rh_slope = 0.0
            lhrh_range = [0,0]

            while not slope_range[0] == slope: #if no convergence
                shot_index = 0

                if point_index == 0: #if there are no points (first loop)
                    for shot in sorted(os.listdir(REF_PATH)): #for each shot
                        out = encode(shot, shot_index, max_range_crf) #encoding
                        assess(shot, out) #quality assessment
                        store_results(shot_index, max_range_crf, out)
                        shot_index += 1
                elif point_index == 1: #if there are no points to compare (second loop)
                    for shot in sorted(os.listdir(REF_PATH)): #for each shot
                        out = encode(shot, shot_index, min_range_crf) #encoding
                        assess(shot, out) #quality assessment
                        store_results(shot_index, min_range_crf, out)
                        shot_index += 1
                    #TODO: first slope
                else:
                    for shot in sorted(os.listdir(REF_PATH)): #for each shot
                        while lh_slope <= slope and rh_slope >= slope and (lhrh_range[1] - lhrh_range[0]) == 2:
                            out = encode(shot, shot_index, o_data["shots"][shot_index]["new_point"]) #encoding
                            assess(shot, out) #quality assessment
                            store_results(shot_index, o_data["shots"][shot_index]["new_point"], out)
                        shot_index += 1
                        save_opt(shot_index, target_index, current_point)

                    #TODO: current RD target
                    #if current_rd > t_val:

                    #TODO: new range
                    #TODO: new slope
                    d = np.array(temp_matrix["durations"])
                    b = np.sum(np.array(temp_matrix["bitrates"]) * d) / np.sum(d)
                    v = np.sum(np.array(temp_matrix["vmafs"]) * d) / np.sum(d)
                    p = np.sum(np.array(temp_matrix["psnrs"]) * d) / np.sum(d)
                    res_matrix = store_results(new_point, b, v, p)


                    #element-wise difference between the metric and its target value
                    difference = np.asarray(abs(np.asarray(res_matrix[t_name]) - t_val))
                    #the minimum difference = the element with the index closer to the target
                    i_first_min = np.argmin(difference)
                    nd_diff = difference.copy()
                    nd_diff[i_first_min] = np.inf #replace the minimum with inf
                    i_second_min = np.argmin(nd_diff) #find the second minimum
                    current_point = i_first_min #the index of the point closer to the target

                    if res_matrix[t_name][i_first_min] == res_matrix[t_name][i_second_min]:
                        #it may happen that the results are the same ex. black short shots
                        current_point = max(i_first_min,i_second_min)
                        new_point = current_point
                    else: #swap the values of the two ends if the lower end is bigger than the upper end
                        if res_matrix[t_name][i_first_min] > res_matrix[t_name][i_second_min]:
                            new_point = search_opt(t_name, t_val, res_matrix[t_name][i_first_min], \
                                        res_matrix[t_name][i_second_min], i_first_min, i_second_min)
                        elif res_matrix[t_name][i_first_min] < res_matrix[t_name][i_second_min]:
                            new_point = search_opt(t_name, t_val, res_matrix[t_name][i_second_min], \
                                        res_matrix[t_name][i_first_min], i_second_min, i_first_min)
                point_index += 1
        target_index += 1
        current_point = None
        new_point = 0
point_index = 0
target_index = 0

ffmpeg version N-106635-g83e1a1de88 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/home/ubuntu/ffmpeg_build --pkg-config-flags=--static --extra-cflags=-I/home/ubuntu/ffmpeg_build/include --extra-ldflags=-L/home/ubuntu/ffmpeg_build/lib --extra-libs='-lpthread -lm' --ld=g++ --bindir=/home/ubuntu/bin --enable-gpl --enable-gnutls --enable-libaom --enable-libass --enable-libfdk-aac --enable-libfreetype --enable-libmp3lame --enable-libopus --enable-libsvtav1 --enable-libdav1d --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libvmaf --enable-version3 --enable-nonfree
  libavutil      57. 24.101 / 57. 24.101
  libavcodec     59. 26.100 / 59. 26.100
  libavformat    59. 22.100 / 59. 22.100
  libavdevice    59.  6.100 / 59.  6.100
  libavfilter     8. 33.100 /  8. 33.100
  libswscale      6.  6.100 /  6.  6.100
  libswresample   4.  6.100 /  4.  6.100
  libpostproc    56.  5.100 / 56.  5.100

frame=  318 fps=244 q=-1.0 Lsize=    9107kB time=00:00:10.51 bitrate=7098.0kbits/s speed=8.08x    
video:9104kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.031043%
[libx264 @ 0x558954ed4180] frame I:2     Avg QP:12.88  size: 80674
[libx264 @ 0x558954ed4180] frame P:269   Avg QP:14.62  size: 32089
[libx264 @ 0x558954ed4180] frame B:47    Avg QP:18.98  size: 11248
[libx264 @ 0x558954ed4180] consecutive B-frames: 77.0%  5.7% 12.3%  5.0%
[libx264 @ 0x558954ed4180] mb I  I16..4:  0.0% 17.3% 82.7%
[libx264 @ 0x558954ed4180] mb P  I16..4:  0.0%  0.2%  0.7%  P16..4: 41.0% 21.8% 36.2%  0.0%  0.0%    skip: 0.1%
[libx264 @ 0x558954ed4180] mb B  I16..4:  0.0%  0.0%  0.0%  B16..8: 11.6%  9.3% 13.7%  direct:42.5%  skip:22.7%  L0:18.2% L1:11.3% BI:70.4%
[libx264 @ 0x558954ed4180] 8x8 transform intra:18.9% inter:22.5%
[libx264 @ 0x558954ed4180] coded y,uvDC,uvAC intra: 100.0% 100.0% 99.9% inter: 87.0% 85.2% 75.6%
[libx264 @ 0x558954ed4180] i8 v,h,dc,ddl,ddr,vr,hd,vl,

frame=  318 fps=146 q=-0.0 Lsize=N/A time=00:00:10.61 bitrate=N/A speed=4.88x    
video:147kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
[Parsed_libvmaf_3 @ 0x55a45048a440] VMAF score: 99.222835
ffmpeg version N-106635-g83e1a1de88 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/home/ubuntu/ffmpeg_build --pkg-config-flags=--static --extra-cflags=-I/home/ubuntu/ffmpeg_build/include --extra-ldflags=-L/home/ubuntu/ffmpeg_build/lib --extra-libs='-lpthread -lm' --ld=g++ --bindir=/home/ubuntu/bin --enable-gpl --enable-gnutls --enable-libaom --enable-libass --enable-libfdk-aac --enable-libfreetype --enable-libmp3lame --enable-libopus --enable-libsvtav1 --enable-libdav1d --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libvmaf --enable-version3 --enable-nonfree
  libavutil      57. 24.101 / 57. 24.101
  libavcodec     59. 26.100 / 59. 26.100
  libavf

frame=  318 fps=274 q=-1.0 Lsize=    6400kB time=00:00:10.51 bitrate=4988.5kbits/s speed=9.07x    
video:6398kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.043443%
[libx264 @ 0x55a94d434180] frame I:2     Avg QP:15.88  size: 67728
[libx264 @ 0x55a94d434180] frame P:269   Avg QP:17.62  size: 22620
[libx264 @ 0x55a94d434180] frame B:47    Avg QP:22.03  size:  7027
[libx264 @ 0x55a94d434180] consecutive B-frames: 77.0%  5.7% 12.3%  5.0%
[libx264 @ 0x55a94d434180] mb I  I16..4:  0.3% 21.8% 77.9%
[libx264 @ 0x55a94d434180] mb P  I16..4:  0.0%  0.2%  0.5%  P16..4: 41.9% 21.3% 34.5%  0.0%  0.0%    skip: 1.6%
[libx264 @ 0x55a94d434180] mb B  I16..4:  0.0%  0.0%  0.0%  B16..8: 12.3%  9.4% 12.1%  direct:35.1%  skip:31.1%  L0:16.3% L1:15.0% BI:68.7%
[libx264 @ 0x55a94d434180] 8x8 transform intra:24.2% inter:21.1%
[libx264 @ 0x55a94d434180] coded y,uvDC,uvAC intra: 100.0% 100.0% 98.6% inter: 76.5% 69.5% 51.3%
[libx264 @ 0x55a94d434180] i16 v,h,dc,p: 33%  0% 67%  

frame=  318 fps=150 q=-0.0 Lsize=N/A time=00:00:10.61 bitrate=N/A speed=   5x    
video:147kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
[Parsed_libvmaf_3 @ 0x55fb46ad5a40] VMAF score: 98.252821
ffmpeg version N-106635-g83e1a1de88 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/home/ubuntu/ffmpeg_build --pkg-config-flags=--static --extra-cflags=-I/home/ubuntu/ffmpeg_build/include --extra-ldflags=-L/home/ubuntu/ffmpeg_build/lib --extra-libs='-lpthread -lm' --ld=g++ --bindir=/home/ubuntu/bin --enable-gpl --enable-gnutls --enable-libaom --enable-libass --enable-libfdk-aac --enable-libfreetype --enable-libmp3lame --enable-libopus --enable-libsvtav1 --enable-libdav1d --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libvmaf --enable-version3 --enable-nonfree
  libavutil      57. 24.101 / 57. 24.101
  libavcodec     59. 26.100 / 59. 26.100
  libavf

...

frame=   90 fps=0.0 q=-1.0 Lsize=      84kB time=00:00:02.90 bitrate= 238.2kbits/s speed=15.5x    
video:83kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 2.264848%
[libx264 @ 0x556f5d924180] frame I:1     Avg QP:38.47  size:  3716
[libx264 @ 0x556f5d924180] frame P:23    Avg QP:39.79  size:  1881
[libx264 @ 0x556f5d924180] frame B:66    Avg QP:43.05  size:   558
[libx264 @ 0x556f5d924180] consecutive B-frames:  1.1%  0.0% 10.0% 88.9%
[libx264 @ 0x556f5d924180] mb I  I16..4:  7.5% 75.7% 16.9%
[libx264 @ 0x556f5d924180] mb P  I16..4:  1.2% 10.7%  1.0%  P16..4: 50.0% 14.4%  4.1%  0.0%  0.0%    skip:18.5%
[libx264 @ 0x556f5d924180] mb B  I16..4:  0.0%  0.3%  0.0%  B16..8: 48.3%  3.8%  0.3%  direct: 0.6%  skip:46.7%  L0:42.9% L1:55.8% BI: 1.3%
[libx264 @ 0x556f5d924180] 8x8 transform intra:81.7% inter:87.0%
[libx264 @ 0x556f5d924180] coded y,uvDC,uvAC intra: 50.1% 44.9% 15.6% inter: 8.3% 4.5% 0.1%
[libx264 @ 0x556f5d924180] i16 v,h,dc,p: 31% 15%  8% 46%
[lib

frame=   90 fps=0.0 q=-0.0 Lsize=N/A time=00:00:03.00 bitrate=N/A speed=5.08x    
video:41kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
[Parsed_libvmaf_3 @ 0x56150dad0600] VMAF score: 60.506424
ffmpeg version N-106635-g83e1a1de88 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/home/ubuntu/ffmpeg_build --pkg-config-flags=--static --extra-cflags=-I/home/ubuntu/ffmpeg_build/include --extra-ldflags=-L/home/ubuntu/ffmpeg_build/lib --extra-libs='-lpthread -lm' --ld=g++ --bindir=/home/ubuntu/bin --enable-gpl --enable-gnutls --enable-libaom --enable-libass --enable-libfdk-aac --enable-libfreetype --enable-libmp3lame --enable-libopus --enable-libsvtav1 --enable-libdav1d --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libvmaf --enable-version3 --enable-nonfree
  libavutil      57. 24.101 / 57. 24.101
  libavcodec     59. 26.100 / 59. 26.100
  libavfo

frame=   90 fps=0.0 q=-1.0 Lsize=      60kB time=00:00:02.90 bitrate= 168.8kbits/s speed=17.6x    
video:58kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 3.226241%
[libx264 @ 0x55ab508ee180] frame I:1     Avg QP:41.50  size:  2683
[libx264 @ 0x55ab508ee180] frame P:23    Avg QP:43.01  size:  1334
[libx264 @ 0x55ab508ee180] frame B:66    Avg QP:46.34  size:   383
[libx264 @ 0x55ab508ee180] consecutive B-frames:  1.1%  0.0% 10.0% 88.9%
[libx264 @ 0x55ab508ee180] mb I  I16..4: 11.4% 81.2%  7.5%
[libx264 @ 0x55ab508ee180] mb P  I16..4:  1.8%  8.4%  0.5%  P16..4: 49.2%  9.5%  3.4%  0.0%  0.0%    skip:27.3%
[libx264 @ 0x55ab508ee180] mb B  I16..4:  0.0%  0.1%  0.0%  B16..8: 42.4%  1.7%  0.1%  direct: 0.3%  skip:55.4%  L0:41.7% L1:57.7% BI: 0.6%
[libx264 @ 0x55ab508ee180] 8x8 transform intra:79.6% inter:92.1%
[libx264 @ 0x55ab508ee180] coded y,uvDC,uvAC intra: 37.1% 39.2% 11.9% inter: 5.2% 3.7% 0.1%
[libx264 @ 0x55ab508ee180] i16 v,h,dc,p: 35% 20% 12% 33%
[lib

frame=   90 fps=0.0 q=-0.0 Lsize=N/A time=00:00:03.00 bitrate=N/A speed=5.04x    
video:41kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: unknown
[Parsed_libvmaf_3 @ 0x55b277a284c0] VMAF score: 47.988549
ffmpeg version N-106635-g83e1a1de88 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/home/ubuntu/ffmpeg_build --pkg-config-flags=--static --extra-cflags=-I/home/ubuntu/ffmpeg_build/include --extra-ldflags=-L/home/ubuntu/ffmpeg_build/lib --extra-libs='-lpthread -lm' --ld=g++ --bindir=/home/ubuntu/bin --enable-gpl --enable-gnutls --enable-libaom --enable-libass --enable-libfdk-aac --enable-libfreetype --enable-libmp3lame --enable-libopus --enable-libsvtav1 --enable-libdav1d --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libvmaf --enable-version3 --enable-nonfree
  libavutil      57. 24.101 / 57. 24.101
  libavcodec     59. 26.100 / 59. 26.100
  libavfo

90
65


## Encode opt video
Put together all the individually encoded shots

In [6]:
def mux(t_i, t_name, t_val):
    file_list = "" #list of encoded vids to be stored in OUT_LIST
    with open(rd_file, 'r') as f:
        o_data = json.load(f)
    for shot in range(0,num_scenes):
        opt_crf = o_data["shots"][0]["opt_points"][t_i]["crf"]
        file_list = file_list + "file '" + DIST_PATH + str(shot) + "/" \
        + str(opt_crf) + "_" + codec.upper() + "." + s_cod["container"] + "' \n"
    with open(OUT_LIST, 'w') as w:
        w.write(file_list)
    o = OUT_PATH+source_name[:9] + "_" + t_name + str(t_val) + "_" + codec.upper() + "." + s_cod["container"]
    mux = f"ffmpeg -f concat -i {OUT_LIST} -c copy {o}"
    subprocess.call(mux, shell=True)

In [7]:
target_index = 0
for t_name in target_list:
    for t_val in target_list[t_name]:
        mux(target_index, t_name, t_val)
        target_index += 1

ffmpeg version N-106635-g83e1a1de88 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/home/ubuntu/ffmpeg_build --pkg-config-flags=--static --extra-cflags=-I/home/ubuntu/ffmpeg_build/include --extra-ldflags=-L/home/ubuntu/ffmpeg_build/lib --extra-libs='-lpthread -lm' --ld=g++ --bindir=/home/ubuntu/bin --enable-gpl --enable-gnutls --enable-libaom --enable-libass --enable-libfdk-aac --enable-libfreetype --enable-libmp3lame --enable-libopus --enable-libsvtav1 --enable-libdav1d --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libvmaf --enable-version3 --enable-nonfree
  libavutil      57. 24.101 / 57. 24.101
  libavcodec     59. 26.100 / 59. 26.100
  libavformat    59. 22.100 / 59. 22.100
  libavdevice    59.  6.100 / 59.  6.100
  libavfilter     8. 33.100 /  8. 33.100
  libswscale      6.  6.100 /  6.  6.100
  libswresample   4.  6.100 /  4.  6.100
  libpostproc    56.  5.100 / 56.  5.100

## TODO: Curve fitting
When the upper search has tested 3 points, given these 3 RQ points, discover the polynomian or logarithmic function that describes their trend. Repeat this when a new point is computed. Measure the error between the approximation and the actual implementation (lagrangian search above) and assess whether and when it may be useful to speed up the search process, by reducing the number of test to encode before the optimum.

NameError: name 'rates' is not defined